<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [47]</a>'.</span>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Description" data-toc-modified-id="Data-Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Description</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Useful-Scripts" data-toc-modified-id="Useful-Scripts-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Useful Scripts</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Data-Processing-and-Feature-Engineering" data-toc-modified-id="Data-Processing-and-Feature-Engineering-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Data Processing and Feature Engineering</a></span><ul class="toc-item"><li><span><a href="#missing-values" data-toc-modified-id="missing-values-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>missing values</a></span></li><li><span><a href="#size-reduction" data-toc-modified-id="size-reduction-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>size reduction</a></span></li><li><span><a href="#Take-date-of-only-2016" data-toc-modified-id="Take-date-of-only-2016-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Take date of only 2016</a></span></li><li><span><a href="#Melt-the-data" data-toc-modified-id="Melt-the-data-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Melt the data</a></span></li><li><span><a href="#Create-date-time-columns" data-toc-modified-id="Create-date-time-columns-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Create date time columns</a></span></li><li><span><a href="#Create-aggregation-columns" data-toc-modified-id="Create-aggregation-columns-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>Create aggregation columns</a></span></li></ul></li><li><span><a href="#String-Manipulation" data-toc-modified-id="String-Manipulation-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>String Manipulation</a></span></li><li><span><a href="#Take-Most-visited-Page-as-timeseries-data" data-toc-modified-id="Take-Most-visited-Page-as-timeseries-data-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Take Most visited Page as timeseries data</a></span></li><li><span><a href="#Save-the-clean-data" data-toc-modified-id="Save-the-clean-data-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Save the clean data</a></span></li><li><span><a href="#Time-Taken" data-toc-modified-id="Time-Taken-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Time Taken</a></span></li></ul></div>

# Data Description
Reference: https://www.kaggle.com/c/web-traffic-time-series-forecasting/data

train_1.csv:
```
rows = 145,063
columns = 551

first column = Page	
date columns = 2015-07-01, 2015-07-02, ..., 2016-12-31 (550 columns)

file size: 284.6 MB



Date columns:
------------------
Jul/2015 - 31 days
Aug/2015 - 31 days
Sep/2015 - 30 days
Oct/2015 - 31 days
Nov/2015 - 30 days
Dec/2015 - 31 days

Total     : 184 days
Year 2016 : 366 days (leap year)
Total     : 550 days

NOTE:
For this dataset, missing data is represented by 0.
```

The training dataset consists of approximately 145k time series. Each of these time series represent a number of daily views of a different Wikipedia article, starting from July, 1st, 2015 up until December 31st, 2016. The leaderboard during the training stage is based on traffic from January, 1st, 2017 up until March 1st, 2017.

The second stage will use training data up until September 1st, 2017. The final ranking of the competition will be based on predictions of daily views between September 13th, 2017 and November 13th, 2017 for each article in the dataset. You will submit your forecasts for these dates by September 12th.

For each time series, you are provided the name of the article as well as the type of traffic that this time series represent (all, mobile, desktop, spider). You may use this metadata and any other publicly available data to make predictions. Unfortunately, the data source for this dataset does not distinguish between traffic values of zero and missing values. A missing value may mean the traffic was zero or that the data is not available for that day.

To reduce the submission file size, each page and date combination has been given a shorter Id. The mapping between page names and the submission Id is given in the key files.


In [1]:
# mkdir ~/.kaggle
# !echo '<PASTE_CONTENTS_OF_KAGGLE_API_JSON>' > ~/.kaggle/kaggle.json

# after we have ~/.kaggle/kaggle.json file in colab, we can install kaggle module.

In [2]:
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!head -c 20 ~/.kaggle/kaggle.json

{"username":"bhishan

In [4]:
%%capture
# capture will not print in notebook

import os
import sys
ENV_COLAB = 'google.colab' in sys.modules

if ENV_COLAB:
    ## install modules
    !pip install watermark
    !pip install fsspec
    !pip install dask[dataframe]

    ## create project like folders
    !mkdir -p ../data ../outputs ../images ../reports ../html ../models

    !pip install kaggle

In [5]:
# !kaggle competitions files -c web-traffic-time-series-forecasting

In [6]:
# !kaggle competitions download -c web-traffic-time-series-forecasting -f train_1.csv.zip -p ../data/

# Imports

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

import os
import time
time_start_notebook = time.time()

# random state
SEED=100
np.random.seed(SEED) # we need this in each cell

# Jupyter notebook settings for pandas
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:,.2g}'.format) # numbers sep by comma
pd.set_option('display.max_rows', 100) # None for all the rows
pd.set_option('display.max_colwidth', 100)

import re
import dask
import dask.dataframe as dd
import gc

# versions
import watermark
%load_ext watermark
%watermark -a "Bhishan Poudel" -d -v -m
print()
%watermark -iv

Bhishan Poudel 2020-10-17 

CPython 3.7.7
IPython 7.18.1

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 19.6.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit

watermark  2.0.2
re         2.2.1
dask       2.13.0
matplotlib 3.2.1
pandas     1.1.0
numpy      1.18.4
seaborn    0.11.0



# Useful Scripts

In [8]:
def show_method_attributes(method, ncols=7):
    """ Show all the attributes of a given method.
    Example:
    ========
    show_method_attributes(list)
     """
    x = [i for i in dir(method) if i[0].islower()]
    x = [i for i in x if i not in 'os np pd sys time psycopg2'.split()]

    return pd.DataFrame(np.array_split(x,ncols)).T.fillna('')

In [9]:
def json_dump_tofile(myjson,ofile,sort_keys=False):
    """Write json dictionary to a datafile.
    
    Usage:
    myjson = {'num': 5, my_list = [1,2,'apple']}
    json_dump_tofile(myjson, ofile)
    
    """
    import io
    import json

    with io.open(ofile, 'w', encoding='utf8') as fo:
        json_str = json.dumps(myjson,
                          indent=4,
                          sort_keys=sort_keys,
                          separators=(',', ': '),
                          ensure_ascii=False)
        fo.write(str(json_str))

# Load the data

In [10]:
!ls ../data

most_visited_2016.csv train_1.csv.zip       train_1_01?raw=true   train_1_02?raw=true   train_1_03?raw=true


In [11]:
df = pd.read_csv('../data/train_1.csv.zip',compression='zip',encoding='latin-1')
print(df.shape)

display(df.head())

(145063, 551)


,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,2015-07-12,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-18,2015-07-19,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-01,2015-08-02,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-08,2015-08-09,2015-08-10,2015-08-11,2015-08-12,2015-08-13,2015-08-14,2015-08-15,2015-08-16,2015-08-17,2015-08-18,...,2016-11-12,2016-11-13,2016-11-14,2016-11-15,2016-11-16,2016-11-17,2016-11-18,2016-11-19,2016-11-20,2016-11-21,2016-11-22,2016-11-23,2016-11-24,2016-11-25,2016-11-26,2016-11-27,2016-11-28,2016-11-29,2016-11-30,2016-12-01,2016-12-02,2016-12-03,2016-12-04,2016-12-05,2016-12-06,2016-12-07,2016-12-08,2016-12-09,2016-12-10,2016-12-11,2016-12-12,2016-12-13,2016-12-14,2016-12-15,2016-12-16,2016-12-17,2016-12-18,2016-12-19,2016-12-20,2016-12-21,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18,11,5,13,14,9,9,22,26,24,19,10,14,15,8,16,8,8,16,7,11,10,20,18,15,14,49,10,16,18,8,5,9,7,13,9,7,4,11,10,5,9,9,9,9,13,4,15,25,...,13,8,15,14,12,6,11,10,42,21,24,14,11,2e+02,14,45,33,28,18,14,47,15,14,18,20,14,16,14,20,60,22,15,17,19,18,21,21,47,65,17,32,63,15,26,14,20,22,19,18,20
1,2PM_zh.wikipedia.org_all-access_spider,11,14,15,18,11,13,22,11,10,4,41,65,57,38,20,62,44,15,10,47,24,17,22,9,39,13,11,12,21,19,9,15,33,8,8,7,13,2,23,12,27,27,36,23,58,80,60,69,42,...,12,11,14,28,23,20,9,12,11,14,14,15,15,11,20,13,19,6.2e+02,57,17,23,19,21,47,28,22,22,65,27,17,17,13,9,18,22,17,15,22,23,19,17,42,28,15,9,30,52,45,26,20
2,3C_zh.wikipedia.org_all-access_spider,1,0,1,1,0,4,0,3,4,4,1,1,1,6,8,6,4,5,1,2,3,8,8,6,6,2,2,3,2,4,3,3,5,3,5,4,2,5,1,4,5,0,0,7,3,5,1,6,2,...,6,4,2,4,6,5,4,4,3,3,9,3,5,4,0,1,4,5,8,8,1,1,2,5,3,3,3,7,3,9,8,3,2.1e+02,5,4,6,2,2,4,3,3,1,1,7,4,4,6,3,4,17
3,4minute_zh.wikipedia.org_all-access_spider,35,13,10,94,4,26,14,9,11,16,16,11,23,1.4e+02,14,17,85,4,30,22,9,10,11,7,7,11,9,11,44,8,14,19,10,17,17,10,7,10,1,8,27,19,16,2,84,22,14,47,25,...,38,13,14,17,26,14,10,9,23,15,7,10,7,10,14,17,11,9,11,5,10,8,17,13,23,40,16,17,41,17,8,9,18,12,12,18,13,18,23,10,32,10,26,27,16,11,17,19,10,11
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,13,11,8,6,10,14,6,9,6,16,14,13,15,14,16,9,1.8e+02,64,12,10,11,6,8,7,9,8,5,11,8,4,15,5,8,8,6,7,15,4,11,7,48,9,25,13,3,11,27,13,36,10


# Data Processing and Feature Engineering

## missing values

In [12]:
# description said zeros and nans are same
df = df.fillna(0)

## size reduction

In [13]:
df.memory_usage(deep=True).sum() * 1e-6 # MB

656.628156

In [14]:
df.iloc[:,1:].max().max()

67264258.0

In [15]:
df.dtypes

Page           object
2015-07-01    float64
2015-07-02    float64
2015-07-03    float64
2015-07-04    float64
               ...   
2016-12-27    float64
2016-12-28    float64
2016-12-29    float64
2016-12-30    float64
2016-12-31    float64
Length: 551, dtype: object

In [16]:
np.iinfo(np.int32).max

2147483647

In [17]:
np.iinfo(np.int32).max > 67264258.0

True

In [18]:
# I can use int32 as datatype

In [19]:
%%time
df.iloc[:,1:] = df.iloc[:,1:].astype(np.int32)

CPU times: user 46.3 s, sys: 36 s, total: 1min 22s
Wall time: 1min 23s


In [20]:
df.memory_usage(deep=True).sum() * 1e-6 # MB

337.489556

## Take date of only 2016

In [21]:
df.iloc[:2, [0,1]]

,Page,2015-07-01
0,2NE1_zh.wikipedia.org_all-access_spider,18
1,2PM_zh.wikipedia.org_all-access_spider,11


In [22]:
t1 = pd.Timestamp('2015-07-01')
t1

Timestamp('2015-07-01 00:00:00')

In [23]:
t2 = pd.Timestamp('2016-01-01')
t2-t1

Timedelta('184 days 00:00:00')

In [24]:
diff = (t2-t1).days
diff

184

In [25]:
df.iloc[:2, [0, diff+1, diff+366, -1]] # 2016 is leap year and iloc is right not-inclusive

,Page,2016-01-01,2016-12-31,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,9,20,20
1,2PM_zh.wikipedia.org_all-access_spider,7,20,20


In [26]:
df = df.iloc[:, np.r_[0,diff+1:diff+1+366]]

df.head()

,Page,2016-01-01,2016-01-02,2016-01-03,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-09,2016-01-10,2016-01-11,2016-01-12,2016-01-13,2016-01-14,2016-01-15,2016-01-16,2016-01-17,2016-01-18,2016-01-19,2016-01-20,2016-01-21,2016-01-22,2016-01-23,2016-01-24,2016-01-25,2016-01-26,2016-01-27,2016-01-28,2016-01-29,2016-01-30,2016-01-31,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,2016-02-11,2016-02-12,2016-02-13,2016-02-14,2016-02-15,2016-02-16,2016-02-17,2016-02-18,...,2016-11-12,2016-11-13,2016-11-14,2016-11-15,2016-11-16,2016-11-17,2016-11-18,2016-11-19,2016-11-20,2016-11-21,2016-11-22,2016-11-23,2016-11-24,2016-11-25,2016-11-26,2016-11-27,2016-11-28,2016-11-29,2016-11-30,2016-12-01,2016-12-02,2016-12-03,2016-12-04,2016-12-05,2016-12-06,2016-12-07,2016-12-08,2016-12-09,2016-12-10,2016-12-11,2016-12-12,2016-12-13,2016-12-14,2016-12-15,2016-12-16,2016-12-17,2016-12-18,2016-12-19,2016-12-20,2016-12-21,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,9,16,6,19,20,19,22,30,14,16,22,15,15,26,16,13,27,18,13,32,31,16,38,18,9,14,10,24,8,15,18,10,23,17,11,26,14,8,12,9,11,34,17,29,11,9,14,21,12,...,13,8,15,14,12,6,11,10,42,21,24,14,11,204,14,45,33,28,18,14,47,15,14,18,20,14,16,14,20,60,22,15,17,19,18,21,21,47,65,17,32,63,15,26,14,20,22,19,18,20
1,2PM_zh.wikipedia.org_all-access_spider,7,15,14,14,11,13,12,12,24,15,38,18,26,15,12,14,40,19,13,39,19,16,19,11,76,14,19,26,19,17,30,17,17,17,19,11,175,10,5,12,7,12,14,19,11,19,17,15,19,...,12,11,14,28,23,20,9,12,11,14,14,15,15,11,20,13,19,621,57,17,23,19,21,47,28,22,22,65,27,17,17,13,9,18,22,17,15,22,23,19,17,42,28,15,9,30,52,45,26,20
2,3C_zh.wikipedia.org_all-access_spider,2,0,3,3,3,4,4,8,3,5,8,1,4,0,3,6,3,1,3,3,3,1,3,8,4,3,2,5,6,3,6,5,6,7,3,1,5,1,2,0,1,4,3,3,9,4,7,5,10,...,6,4,2,4,6,5,4,4,3,3,9,3,5,4,0,1,4,5,8,8,1,1,2,5,3,3,3,7,3,9,8,3,210,5,4,6,2,2,4,3,3,1,1,7,4,4,6,3,4,17
3,4minute_zh.wikipedia.org_all-access_spider,7,7,11,7,14,9,21,9,10,13,10,13,16,8,10,7,13,18,8,50,8,33,6,22,9,84,28,11,7,14,16,49,71,29,22,6,34,16,14,9,12,24,18,8,26,8,8,13,21,...,38,13,14,17,26,14,10,9,23,15,7,10,7,10,14,17,11,9,11,5,10,8,17,13,23,40,16,17,41,17,8,9,18,12,12,18,13,18,23,10,32,10,26,27,16,11,17,19,10,11
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,13,11,8,6,10,14,6,9,6,16,14,13,15,14,16,9,178,64,12,10,11,6,8,7,9,8,5,11,8,4,15,5,8,8,6,7,15,4,11,7,48,9,25,13,3,11,27,13,36,10


In [27]:
df = df.sample(frac=0.1,random_state=SEED)

## Melt the data

In [28]:
df = df.melt(id_vars=['Page'],var_name='date',value_name='visits')

print(df.shape)
df.head()

(5309196, 3)


,Page,date,visits
0,Sean_Connery_en.wikipedia.org_desktop_all-agents,2016-01-01,4872
1,Tableau_des_mÃ©dailles_des_Jeux_olympiques_d'Ã©tÃ©_de_2008_fr.wikipedia.org_desktop_all-agents,2016-01-01,6
2,The_Undertaker_fr.wikipedia.org_mobile-web_all-agents,2016-01-01,469
3,Category:Outdoor_sex_commons.wikimedia.org_all-access_all-agents,2016-01-01,142
4,ÐÐ°Ð¼ÑÐ·ÑÐº_ru.wikipedia.org_all-access_all-agents,2016-01-01,6692


In [29]:
df['date'] = pd.to_datetime(df['date'])

In [30]:
df.dtypes

Page              object
date      datetime64[ns]
visits             int32
dtype: object

## Create date time columns

In [31]:
show_method_attributes(df['date'].dt)

,0,1,2,3,4,5,6
0,ceil,days_in_month,is_month_end,isocalendar,normalize,timetz,tz_localize
1,date,daysinmonth,is_month_start,microsecond,quarter,to_period,week
2,day,floor,is_quarter_end,minute,round,to_pydatetime,weekday
3,day_name,freq,is_quarter_start,month,second,tz,weekofyear
4,dayofweek,hour,is_year_end,month_name,strftime,tz_convert,year
5,dayofyear,is_leap_year,is_year_start,nanosecond,,,


In [32]:
%%time

# we can make these values categorical to reduce memory,
# but later for modelling, we need numpy array.
df['year'] = df['date'].dt.year # yyyy
df['month'] = df['date'].dt.month # 1 to 12
df['day'] = df['date'].dt.day # 1 to 31

df['quarter'] = df['date'].dt.quarter # 1 to 4
df['dayofweek'] = df['date'].dt.dayofweek # 0 to 6
df['dayofyear'] = df['date'].dt.dayofyear # 1 to 366 (leap year)

df['day_name'] = df['date'].dt.day_name() # Monday
df['month_name'] = df['date'].dt.month_name() # January

df['weekend'] = ((df['date'].dt.dayofweek) // 5 == 1)
df['weekday'] = ((df['date'].dt.dayofweek) // 5 != 1)

CPU times: user 3.2 s, sys: 247 ms, total: 3.45 s
Wall time: 3.49 s


In [33]:
df.head(20)

,Page,date,visits,year,month,day,quarter,dayofweek,dayofyear,day_name,month_name,weekend,weekday
0,Sean_Connery_en.wikipedia.org_desktop_all-agents,2016-01-01,4872,2016,1,1,1,4,1,Friday,January,False,True
1,Tableau_des_mÃ©dailles_des_Jeux_olympiques_d'Ã©tÃ©_de_2008_fr.wikipedia.org_desktop_all-agents,2016-01-01,6,2016,1,1,1,4,1,Friday,January,False,True
2,The_Undertaker_fr.wikipedia.org_mobile-web_all-agents,2016-01-01,469,2016,1,1,1,4,1,Friday,January,False,True
3,Category:Outdoor_sex_commons.wikimedia.org_all-access_all-agents,2016-01-01,142,2016,1,1,1,4,1,Friday,January,False,True
4,ÐÐ°Ð¼ÑÐ·ÑÐº_ru.wikipedia.org_all-access_all-agents,2016-01-01,6692,2016,1,1,1,4,1,Friday,January,False,True
5,File:PioneerSodHouse-WheatRidgeCO.jpg_commons.wikimedia.org_desktop_all-agents,2016-01-01,0,2016,1,1,1,4,1,Friday,January,False,True
6,ÐÐµÐ¶Ð´ÑÐ½Ð°ÑÐ¾Ð´Ð½Ð°Ñ_ÐºÐ¾ÑÐ¼Ð¸ÑÐµÑÐºÐ°Ñ_ÑÑÐ°Ð½ÑÐ¸Ñ_ru.wikipedia.org_all-access_spider,2016-01-01,33,2016,1,1,1,4,1,Friday,January,False,True
7,Volleyball_at_the_2016_Summer_Olympics_â_Men's_tournament_en.wikipedia.org_all-access_all-agents,2016-01-01,123,2016,1,1,1,4,1,Friday,January,False,True
8,Marianne_James_fr.wikipedia.org_all-access_all-agents,2016-01-01,424,2016,1,1,1,4,1,Friday,January,False,True
9,é ç«¥MJ116_zh.wikipedia.org_desktop_all-agents,2016-01-01,254,2016,1,1,1,4,1,Friday,January,False,True


## Create aggregation columns

In [34]:
%%time
df['mean'] = df.groupby('Page')['visits'].transform('mean')
df['median'] = df.groupby('Page')['visits'].transform('median')

CPU times: user 2.18 s, sys: 68.1 ms, total: 2.25 s
Wall time: 2.27 s


# String Manipulation

Example of one page:
```
2NE1_zh.wikipedia.org_all-access_spider
name_project_access_agent

```

In [35]:
df['Page'].head(10)

0                                                       Sean_Connery_en.wikipedia.org_desktop_all-agents
1         Tableau_des_mÃ©dailles_des_Jeux_olympiques_d'Ã©tÃ©_de_2008_fr.wikipedia.org_desktop_all-agents
2                                                  The_Undertaker_fr.wikipedia.org_mobile-web_all-agents
3                                       Category:Outdoor_sex_commons.wikimedia.org_all-access_all-agents
4                                                  ÐÐ°Ð¼ÑÐ·ÑÐº_ru.wikipedia.org_all-access_all-agents
5                         File:PioneerSodHouse-WheatRidgeCO.jpg_commons.wikimedia.org_desktop_all-agents
6    ÐÐµÐ¶Ð´ÑÐ½Ð°ÑÐ¾Ð´Ð½Ð°Ñ_ÐºÐ¾ÑÐ¼Ð¸ÑÐµÑÐºÐ°Ñ_ÑÑÐ°Ð½ÑÐ¸Ñ_ru.wikipedia.org_all-access_spider
7     Volleyball_at_the_2016_Summer_Olympics_â_Men's_tournament_en.wikipedia.org_all-access_all-agents
8                                                  Marianne_James_fr.wikipedia.org_all-access_all-agents
9                                                      

In [36]:
regex = ( r'(.+)_'  # name
          r'(.+)_'  # project
          r'(.+)_'  # access
          r'(.+)'   # agent
        )

lang_map ={'en':'English','ja':'Japanese','de':'German',
        'www':'Media','fr':'French','zh':'Chinese',
        'ru':'Russian','es':'Spanish','commons': 'Media'
       }

# another way
#
# df['agent'] = df['page'].str.split('_').str[-1]
# df['access'] = df['page'].str.split('_').str[-2]
# df['project'] = df['page'].str.split('_').str[-3]
# df['name'] = df['page'].str.split('_').str[:-3].str.join('_')

def myfunc(df):
    df = df.copy()
    df[['name','project','access','agent']]  = df['Page'].str.extract(regex,expand=True)
    df['lang'] = df['project'].str.split('.').str[0]
    df['language'] = df['lang'].map(lang_map)

    return df

In [37]:
import dask.dataframe as dd
import gc

ddf = dd.from_pandas(df, npartitions=40)

def dask_apply(): 
    return ddf.map_partitions(myfunc).compute()

df = dask_apply()

del ddf
gc.collect()

df.head()

,Page,date,visits,year,month,day,quarter,dayofweek,dayofyear,day_name,month_name,weekend,weekday,mean,median,name,project,access,agent,lang,language
0,Sean_Connery_en.wikipedia.org_desktop_all-agents,2016-01-01,4872,2016,1,1,1,4,1,Friday,January,False,True,3.4e+03,2.6e+03,Sean_Connery,en.wikipedia.org,desktop,all-agents,en,English
1,Tableau_des_mÃ©dailles_des_Jeux_olympiques_d'Ã©tÃ©_de_2008_fr.wikipedia.org_desktop_all-agents,2016-01-01,6,2016,1,1,1,4,1,Friday,January,False,True,1.7e+02,18,Tableau_des_mÃ©dailles_des_Jeux_olympiques_d'Ã©tÃ©_de_2008,fr.wikipedia.org,desktop,all-agents,fr,French
2,The_Undertaker_fr.wikipedia.org_mobile-web_all-agents,2016-01-01,469,2016,1,1,1,4,1,Friday,January,False,True,4e+02,3.5e+02,The_Undertaker,fr.wikipedia.org,mobile-web,all-agents,fr,French
3,Category:Outdoor_sex_commons.wikimedia.org_all-access_all-agents,2016-01-01,142,2016,1,1,1,4,1,Friday,January,False,True,2.1e+02,1.9e+02,Category:Outdoor_sex,commons.wikimedia.org,all-access,all-agents,commons,Media
4,ÐÐ°Ð¼ÑÐ·ÑÐº_ru.wikipedia.org_all-access_all-agents,2016-01-01,6692,2016,1,1,1,4,1,Friday,January,False,True,9.1e+02,5.6e+02,ÐÐ°Ð¼ÑÐ·ÑÐº,ru.wikipedia.org,all-access,all-agents,ru,Russian


In [38]:
for x in ['project','agent','access','lang']:
    print(x)
    print(df[x].value_counts())
    print()

project
en.wikipedia.org         863028
ja.wikipedia.org         756156
de.wikipedia.org         688080
fr.wikipedia.org         637938
zh.wikipedia.org         637572
ru.wikipedia.org         566568
es.wikipedia.org         506544
commons.wikimedia.org    390888
www.mediawiki.org        262422
Name: project, dtype: int64

agent


all-agents    4050156
spider        1259040
Name: agent, dtype: int64

access
all-access    2705106
mobile-web    1331874
desktop       1272216
Name: access, dtype: int64

lang


en         863028
ja         756156
de         688080
fr         637938
zh         637572
ru         566568
es         506544
commons    390888
www        262422
Name: lang, dtype: int64



# Take Most visited Page as timeseries data

In [39]:
df.head(2)

,Page,date,visits,year,month,day,quarter,dayofweek,dayofyear,day_name,month_name,weekend,weekday,mean,median,name,project,access,agent,lang,language
0,Sean_Connery_en.wikipedia.org_desktop_all-agents,2016-01-01,4872,2016,1,1,1,4,1,Friday,January,False,True,3.4e+03,2.6e+03,Sean_Connery,en.wikipedia.org,desktop,all-agents,en,English
1,Tableau_des_mÃ©dailles_des_Jeux_olympiques_d'Ã©tÃ©_de_2008_fr.wikipedia.org_desktop_all-agents,2016-01-01,6,2016,1,1,1,4,1,Friday,January,False,True,1.7e+02,18,Tableau_des_mÃ©dailles_des_Jeux_olympiques_d'Ã©tÃ©_de_2008,fr.wikipedia.org,desktop,all-agents,fr,French


In [40]:
# top pages per language
df.groupby('language')['visits'].apply(lambda x: df.loc[x.nlargest(1).index])

,,Page,date,visits,year,month,day,quarter,dayofweek,dayofyear,day_name,month_name,weekend,weekday,mean,median,name,project,access,agent,lang,language
language,,,,,,,,,,,,,,,,,,,,,,
Chinese,3526717,ç·å½çäºèäºè§è²åè¡¨_zh.wikipedia.org_desktop_all-agents,2016-08-31,243557,2016,8,31,3,2,244,Wednesday,August,False,True,8.1e+02,1.3e+02,ç·å½çäºèäºè§è²åè¡¨,zh.wikipedia.org,desktop,all-agents,zh,Chinese
English,2714919,Special:Search_en.wikipedia.org_desktop_all-agents,2016-07-06,16592075,2016,7,6,3,2,188,Wednesday,July,False,True,1.8e+06,1.7e+06,Special:Search,en.wikipedia.org,desktop,all-agents,en,English
French,2163034,WikipÃ©dia:Accueil_principal_fr.wikipedia.org_all-access_all-agents,2016-05-29,1845404,2016,5,29,2,6,150,Sunday,May,True,False,1.6e+06,1.6e+06,WikipÃ©dia:Accueil_principal,fr.wikipedia.org,all-access,all-agents,fr,French
German,4439792,GerÃ¤testecker_de.wikipedia.org_desktop_all-agents,2016-11-02,558381,2016,11,2,4,2,307,Wednesday,November,False,True,1.9e+03,4e+02,GerÃ¤testecker,de.wikipedia.org,desktop,all-agents,de,German
Japanese,2724137,ãã¤ã´ã£ããã»ã­ãã¯ãã§ã©ã¼_ja.wikipedia.org_all-access_all-agents,2016-07-06,1651272,2016,7,6,3,2,188,Wednesday,July,False,True,8e+03,1.4e+02,ãã¤ã´ã£ããã»ã­ãã¯ãã§ã©ã¼,ja.wikipedia.org,all-access,all-agents,ja,Japanese
Media,1865460,Parsoid/Developer_Setup_www.mediawiki.org_all-access_all-agents,2016-05-08,927825,2016,5,8,2,6,129,Sunday,May,True,False,1.3e+04,60,Parsoid/Developer_Setup,www.mediawiki.org,all-access,all-agents,www,Media
Russian,4179962,Ð¡Ð»ÑÐ¶ÐµÐ±Ð½Ð°Ñ:ÐÐ¾Ð¸ÑÐº_ru.wikipedia.org_all-access_all-agents,2016-10-15,1412292,2016,10,15,4,5,289,Saturday,October,True,False,1.8e+05,1.7e+05,Ð¡Ð»ÑÐ¶ÐµÐ±Ð½Ð°Ñ:ÐÐ¾Ð¸ÑÐº,ru.wikipedia.org,all-access,all-agents,ru,Russian
Spanish,2376682,Nilo_es.wikipedia.org_desktop_all-agents,2016-06-12,783454,2016,6,12,2,6,164,Sunday,June,True,False,2.8e+03,6.3e+02,Nilo,es.wikipedia.org,desktop,all-agents,es,Spanish


In [41]:
idx = df.groupby('Page')['visits'].sum().idxmax()
idx

'Special:Search_en.wikipedia.org_desktop_all-agents'

In [42]:
df.query(""" Page == @idx """).head()

,Page,date,visits,year,month,day,quarter,dayofweek,dayofyear,day_name,month_name,weekend,weekday,mean,median,name,project,access,agent,lang,language
2297,Special:Search_en.wikipedia.org_desktop_all-agents,2016-01-01,1401667,2016,1,1,1,4,1,Friday,January,False,True,1.8e+06,1.7e+06,Special:Search,en.wikipedia.org,desktop,all-agents,en,English
16803,Special:Search_en.wikipedia.org_desktop_all-agents,2016-01-02,1395136,2016,1,2,1,5,2,Saturday,January,True,False,1.8e+06,1.7e+06,Special:Search,en.wikipedia.org,desktop,all-agents,en,English
31309,Special:Search_en.wikipedia.org_desktop_all-agents,2016-01-03,1455522,2016,1,3,1,6,3,Sunday,January,True,False,1.8e+06,1.7e+06,Special:Search,en.wikipedia.org,desktop,all-agents,en,English
45815,Special:Search_en.wikipedia.org_desktop_all-agents,2016-01-04,1750373,2016,1,4,1,0,4,Monday,January,False,True,1.8e+06,1.7e+06,Special:Search,en.wikipedia.org,desktop,all-agents,en,English
60321,Special:Search_en.wikipedia.org_desktop_all-agents,2016-01-05,1787494,2016,1,5,1,1,5,Tuesday,January,False,True,1.8e+06,1.7e+06,Special:Search,en.wikipedia.org,desktop,all-agents,en,English


In [43]:
# I am using page other than special search.

df.query('lang == "en"').nlargest(5,'visits')

,Page,date,visits,year,month,day,quarter,dayofweek,dayofyear,day_name,month_name,weekend,weekday,mean,median,name,project,access,agent,lang,language
2714919,Special:Search_en.wikipedia.org_desktop_all-agents,2016-07-06,16592075,2016,7,6,3,2,188,Wednesday,July,False,True,1.8e+06,1.7e+06,Special:Search,en.wikipedia.org,desktop,all-agents,en,English
3556267,Special:Search_en.wikipedia.org_desktop_all-agents,2016-09-02,7599524,2016,9,2,3,4,246,Friday,September,False,True,1.8e+06,1.7e+06,Special:Search,en.wikipedia.org,desktop,all-agents,en,English
3570773,Special:Search_en.wikipedia.org_desktop_all-agents,2016-09-03,6894531,2016,9,3,3,5,247,Saturday,September,True,False,1.8e+06,1.7e+06,Special:Search,en.wikipedia.org,desktop,all-agents,en,English
3541761,Special:Search_en.wikipedia.org_desktop_all-agents,2016-09-01,6878515,2016,9,1,3,3,245,Thursday,September,False,True,1.8e+06,1.7e+06,Special:Search,en.wikipedia.org,desktop,all-agents,en,English
3585279,Special:Search_en.wikipedia.org_desktop_all-agents,2016-09-04,6457072,2016,9,4,3,6,248,Sunday,September,True,False,1.8e+06,1.7e+06,Special:Search,en.wikipedia.org,desktop,all-agents,en,English


In [44]:
df.query('lang == "en" and name != "Special:Search" ').nlargest(5,'visits')

,Page,date,visits,year,month,day,quarter,dayofweek,dayofyear,day_name,month_name,weekend,weekday,mean,median,name,project,access,agent,lang,language
801124,Web_scraping_en.wikipedia.org_all-access_all-agents,2016-02-25,4656065,2016,2,25,1,3,56,Thursday,February,False,True,1.1e+05,9.6e+02,Web_scraping,en.wikipedia.org,all-access,all-agents,en,English
194725,Alan_Rickman_en.wikipedia.org_all-access_all-agents,2016-01-14,3402109,2016,1,14,1,3,14,Thursday,January,False,True,2.7e+04,1e+04,Alan_Rickman,en.wikipedia.org,all-access,all-agents,en,English
757606,Web_scraping_en.wikipedia.org_all-access_all-agents,2016-02-22,3337999,2016,2,22,1,0,53,Monday,February,False,True,1.1e+05,9.6e+02,Web_scraping,en.wikipedia.org,all-access,all-agents,en,English
1624114,Prince_(musician)_en.wikipedia.org_mobile-web_all-agents,2016-04-21,3320724,2016,4,21,2,3,112,Thursday,April,False,True,3.7e+04,5.3e+03,Prince_(musician),en.wikipedia.org,mobile-web,all-agents,en,English
1638620,Prince_(musician)_en.wikipedia.org_mobile-web_all-agents,2016-04-22,3290304,2016,4,22,2,4,113,Friday,April,False,True,3.7e+04,5.3e+03,Prince_(musician),en.wikipedia.org,mobile-web,all-agents,en,English


In [45]:
# Now I see some interesting pages such as webscraping, allan-rickman and prince.
# I will take prince as the timeseries data for modelling.

In [46]:
df.head(2)

,Page,date,visits,year,month,day,quarter,dayofweek,dayofyear,day_name,month_name,weekend,weekday,mean,median,name,project,access,agent,lang,language
0,Sean_Connery_en.wikipedia.org_desktop_all-agents,2016-01-01,4872,2016,1,1,1,4,1,Friday,January,False,True,3.4e+03,2.6e+03,Sean_Connery,en.wikipedia.org,desktop,all-agents,en,English
1,Tableau_des_mÃ©dailles_des_Jeux_olympiques_d'Ã©tÃ©_de_2008_fr.wikipedia.org_desktop_all-agents,2016-01-01,6,2016,1,1,1,4,1,Friday,January,False,True,1.7e+02,18,Tableau_des_mÃ©dailles_des_Jeux_olympiques_d'Ã©tÃ©_de_2008,fr.wikipedia.org,desktop,all-agents,fr,French


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [47]:
cond = train['Page'].str.lower().str.startswith('prince_(musician)_en')

df_prince = train.loc[cond]

df_prince.head()

NameError: name 'train' is not defined

In [ ]:
cond = df['Page'] == "Prince_(musician)_en.wikipedia.org_all-access_all-agents"

df_prince = df.loc[cond]

df_prince.head()

# this is one row, i need to melt it later.

In [ ]:
df_prince.filter(regex="Page|2016").iloc[:5, np.r_[0, 1,2,-2,-1]]

# Save the clean data

In [ ]:
# %%time
# df.to_csv('../data/data_cleaned_2016_sample.csv',index=False)

# 1.14 GB, cant be uploaded to github.

# CPU times: user 2min 15s, sys: 2.14 s, total: 2min 17s
# Wall time: 2min 36s

# pandas is faster to write file.

In [ ]:
# %%time
# ddf = dd.from_pandas(df, npartitions=40)

# ddf.to_csv('../../data/wiki/processed/wikipedia_2016_frac01.csv',index=False)

# CPU times: user 2min 34s, sys: 3.74 s, total: 2min 38s
# Wall time: 2min 45s

# Time Taken

In [ ]:
time_taken = time.time() - time_start_notebook
h,m = divmod(time_taken,60*60)
print('Time taken to run whole notebook: {:.0f} hr '\
      '{:.0f} min {:.0f} secs'.format(h, *divmod(m,60)))